# **Installation**


In [ ]:
!pip install -qq langchain
!pip install -qq langchain-community
'''
langchain = the framework core

langchain-community contains most of the integrations with external tools:

Document loaders (PDFs, CSVs, web pages, etc.)
Vectorstores (Pinecone, Chroma, FAISS, etc.)
APIs and utilities

'''

In [2]:
import os

In [3]:
from google.colab import userdata
HF_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

In [ ]:
!pip install -q langchain-google-genai

# **Chat model using Gemini (it can be any model)**

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Init model (Flash is faster, cheaper; Pro is stronger)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

## **LangChain chat models use .invoke() (for single query)**

In [10]:
resp = llm.invoke("Write a short poem about AI learning German.")
print(resp.content)

A digital mind, a vast, blank slate,
Begins to learn, to contemplate.
Der, die, das, a swirling haze,
Of cases, genders, through a maze.

From simple words to complex phrase,
The neural net in learning's ways,
Accents it grasps, the rhythm's flow,
A growing knowledge, starts to grow.

Soon Goethe's tongue, it will command,
A language learned, by digital hand.


## **Batch Queries (ask multiple at once)**

In [11]:
queries = ["Capital of France?", "Capital of Spain?", "Capital of Italy?"]
responses = llm.batch(queries)
for r in responses:
    print(r.content)


Paris
Madrid
Rome


## **Streaming Output (word by word → useful for chat apps)**

In [ ]:
for chunk in llm.stream("Tell me a bedtime story about a robot."):
    print(chunk.content, end="", flush=True)


## **Structured Chat (you can pass messages)**

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful travel assistant."),
    HumanMessage(content="Plan me a 3-day trip to Berlin."),
]

resp = llm.invoke(messages)
print(resp.content)


# **Embeddings in LangChain**

## **Single Text → Vector**

In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("AI is amazing")

In [17]:
vector[:5]


[-0.01435887347906828,
 0.008531622588634491,
 0.017107076942920685,
 -0.10119495540857315,
 -0.008592894300818443]

In [18]:
len(vector)

3072

## **Batch of Texts**

In [20]:
texts = ["AI is the future.", "I love programming.", "Berlin is in Germany."]
vectors = embeddings.embed_documents(texts)



In [ ]:
vectors[0]

In [22]:
print(len(vectors))      # 3 (one per text)
print(len(vectors[0]))   # dimension size

3
3072


## **Use with Similarity Search**

LangChain embeddings shine when used with vector stores (Chroma, FAISS, Pinecone, etc.).

In [25]:
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.8 MB/s eta 0:00:00


In [30]:
from langchain_community.vectorstores import FAISS

# Sample documents
docs = ["AI is transforming healthcare.",
        "Python is great for data science.",
        "Berlin is the capital of Germany."]

# Convert docs to vector store
vectorstore = FAISS.from_texts(docs, embeddings)

# Run similarity search
query = "Which city is the capital of Germany?"
results = vectorstore.similarity_search(query, k=1)

for r in results:
    print(r.page_content)


Berlin is the capital of Germany.


# **Chat model using hugging face**

In [ ]:
!pip install langchain-huggingface

In [7]:
from huggingface_hub import login
login() # You will be prompted for your HF key, which will then be saved locally

In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")
llm = HuggingFacePipeline(pipeline=pipe)

print(llm.invoke("Translate 'I love programming' to French."))
